In [11]:
%config Completer.use_jedi = False

In [1]:
import pandas as pd
import jupyter_dash

In [56]:
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output


In [51]:
import combine_data
import utils

In [31]:
from importlib import reload

In [76]:
reload(combine_data)

data = combine_data.get_csv_and_navest()

In [75]:
data = data.astype({
    col:'float'
    for col 
    in ['temperature', 'salinity_y', 'conductivity_y', 'pressure_y', 'obs_proj', 'oxygen_proj', 'depth']
})

In [37]:
columns = list(data.columns)

# ST Plot

In [47]:
def create_selector(html_id, label, default):
    return html.Div(children=[
            label,  
            dcc.Dropdown(columns, default, id=html_id, style={'padding-left':'5px', 'width': '60%'})
        ], style={'display': 'flex', 'flex-direction':'row'})

In [121]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

import plotly.express as px
import pandas as pd

app = JupyterDash(__name__)

app.layout = html.Div(children=[
    html.Div(children=[
        create_selector('x_axis', "X Axis", 'temperature'),
        create_selector('y_axis', "Y Axis", 'salinity_y'),
        create_selector('color', "Color", 'pressure_y'),
        
    
    ]), #, style={'display': 'flex', 'flex-direction': 'row'}

    dcc.Graph(
        id='st-plot',
    )
])

@app.callback(
    Output(component_id='st-plot', component_property='figure'),
    [
        Input(component_id='x_axis', component_property='value'),
        Input(component_id='y_axis', component_property='value'),
        Input(component_id='color', component_property='value')
    ]
)
def st_plot(x_input, y_input, color_input):
    return px.scatter(
        data,
        x_input,
        y_input,
        color=color_input
    )

if __name__ == '__main__':
    app.run_server('inline')


# Spatial plot

In [120]:
mesh = utils.get_mesh(utils.MESH_PATH, 1000)

app = JupyterDash(__name__)

app.layout = html.Div(children=[
    dcc.Dropdown(columns, 'temperature', id='color_selector'),
    dcc.Slider(min = -2000, max=-1500, value=-1700, id='min_depth'),
    dcc.Graph(id = 'spatial')
])

@app.callback(
    Output(component_id = 'spatial', component_property='figure'),
    [
        Input(component_id = 'color_selector', component_property='value'),
        Input(component_id='min_depth', component_property='value')
    ]
)
def update_spatial_graph(color_col, min_depth):
    indices = data['depth'] < min_depth
    
    scatter = go.Scatter3d(
        x = data[indices].northing, 
        y = data[indices].easting, 
        z = data[indices].depth, 
        mode = 'markers',
        marker = dict(
            color = data[indices][color_col], 
            colorscale = 'turbo',
            colorbar=dict(thickness=15, outlinewidth=0)
        )
    )
    fig = go.Figure(data=[mesh, scatter])
    fig.update_layout(
        margin=dict(l=20, r=20, t=40, b=40),
    )
    
    return fig

app.run_server('inline')